In [1]:
import numpy as np

In [16]:
def uunifastdiscard(n,ut,discard_limit = 1000):
    sumu = ut
    for i in range(discard_limit):
        utilities = []
        for i in range(n-1):
            nextsumu = sumu*(np.random.random()**(1/(n-1)))
            utilities.append(sumu - nextsumu)
            sumu = nextsumu

        utilities.append(sumu)

        if all(ut <= 1 for ut in utilities):
            return np.array(utilities)
        else:
            sumu = ut

    return False
            

In [15]:
def periods_generation(n,min_period,max_period):
    periods = np.ceil(np.exp(np.random.uniform(low=np.log(min_period), high=np.log(max_period),size=(n))))
    return periods

In [20]:
def execution_times(utilities,periods):
    return np.ceil(utilities * periods)

In [18]:
def task_deadlines(execution_time,period):
    deadline = []
    for i in range(len(execution_time)):
        deadline.append(np.random.uniform(low = execution_time[i] , high = period[i]))

    return np.floor(np.array(deadline))

In [29]:
n = 10
u = 5
def taskset_generation(n,u):
    ut = uunifastdiscard(n,u)
    p = periods_generation(10,1,1000)
    e = execution_times(ut,p)
    d = task_deadlines(e,p)
    taskset = [[p[i],e[i],d[i]] for i in range(len(p))]

taskset_generation(10,5)


## Schedulability tests

In [15]:
def deadline_analysis(ptaskset,m):    #taskset = {period,execution time,deadline}
    num_tasks = len(ptaskset)
    for i in range(num_tasks):
        total_workload = 0
        ei,di = ptaskset[i][1],ptaskset[i][2]
        l = di
        for j in range(i):
            pj,ej,dj = ptaskset[j][0],ptaskset[j][1],ptaskset[j][2]
            max_jobs = np.floor(( l + dj - ej )/pj)
            workload_j = max_jobs*ej + min(ej,l + dj - ej - max_jobs*pj)
            workload_j = min(workload_j,di - ei + 1)
            total_workload += workload_j

        thresh = ei + np.floor(total_workload/m)
        if di < thresh:
            return 0
    return 1

In [14]:
def responsetimeanalysis(ptaskset,m):
    num_tasks = len(ptaskset)
    for i in range(num_tasks):
        ei,di = ptaskset[i][1],ptaskset[i][2]
        l = di
        res = ei
        new_res = ei + 10
        if i == 1:
            if res <= di:
                continue
            else:
                return 0

        while new_res - res > 0.7 and new_res <= di :

            total_workload = 0
            for j in range(i):
                pj,ej = ptaskset[j][0],ptaskset[j][1]
                max_jobs = np.floor(( l + res - ej )/pj)
                workload_j = max_jobs*ej + min(ej,l + res - ej - max_jobs*pj)
                workload_j = min(workload_j,res - ei + 1)
                total_workload += workload_j

            new_res = ei + np.floor(total_workload/m)

        if new_res - res <= 0.7 and new_res <= di :
            continue
        else:
            return 0

    return 1

## Scheduling algorithms

In [26]:
def DMPO(taskset):
    taskset.sort(key = lambda x:x[-1])

In [33]:
def D_CMPO(taskset):
    taskset.sort(key = lambda x:x[-1] - x[-2])

In [37]:
def DkC(taskset,m):
    k = ((m - 1) + np.sqrt(5*m**2 - 6*m + 1))/(2*m)
    taskset.sort(key = lambda x:x[-1] - k*x[-2])    

In [ ]:
def OPA(taskset,m):
    ## total concepts and final code
    print("do")

# start

In [2]:
class Task():
    def __init__(self, period, execution_time, deadline ):
        self.period = period
        self.execution_time = execution_time
        self.deadline = deadline
        self.priority = None

    def __repr__(self) -> str:
        return "Task: " +  str(self.period) +" " + str(self.execution_time) + " " + str(self.deadline) + " " + str(self.priority)

class TaskSet():

    def __init__(self, n, ut, min_period, max_period):
        self.n = n
        self.ut = ut
        self.min_period = min_period
        self.max_period = max_period
        self.tasks = None

    def generate(self):
        self.utilities = self.uunifastdiscard()
        if self.utilities is None:
            print("Failed to generate taskset with given parameters")
            return False

        self.periods = self.periods_generation()
        self.execution_time = self.execution_times()
        self.deadline = self.task_deadlines()

        self.tasks = []
        for i in range(self.n):
            self.tasks.append(Task(self.periods[i],self.execution_time[i],self.deadline[i]))
        return True
        

    def uunifastdiscard(self,discard_limit = 1000):
        sumu = self.ut
        for j in range(discard_limit):
            utilities = []
            for i in range(self.n-1):
                nextsumu = sumu*(np.random.random()**(1/(self.n-i+1)))
                utilities.append(sumu - nextsumu)
                sumu = nextsumu

            utilities.append(sumu)

            if all(ut <= 1 for ut in utilities):
                return np.array(utilities)
            else:
                sumu = self.ut

        return None
    
    def periods_generation(self):
        periods = np.ceil(np.exp(np.random.uniform(low=np.log(self.min_period), high=np.log(self.max_period),size=(self.n))))
        return periods
    
    def execution_times(self):
        return np.ceil(self.utilities * self.periods)
    
    def task_deadlines(self):
        deadline = []
        for i in range(len(self.execution_time)):
            deadline.append(np.random.uniform(low = self.execution_time[i] , high = self.periods[i]))

        return np.floor(np.array(deadline))

In [5]:
def DMPO(set):
    set.sort(key = lambda x:x.deadline)

In [124]:
def D_CMPO(taskset):
    taskset.sort(key = lambda x:x.deadline - x.execution_time)

In [128]:
def DkC(taskset,m):
    k = ((m - 1) + np.sqrt(5*m**2 - 6*m + 1))/(2*m)
    taskset.sort(key = lambda x:x.deadline - k*x.execution_time)

In [6]:
def deadline_analysis_OPA(ptaskset,m):    #taskset = {period,execution time,deadline}
    Ck = ptaskset[-1].execution_time
    Dk = ptaskset[-1].deadline
    l = Dk
    total_workload = 0

    for task in ptaskset[:-1]:
        Ti,Ci,Di = task.period,task.execution_time,task.deadline
        max_jobs = np.floor(( l + Di - Ci )/Ti)
        workload_task = max_jobs*Ci + min(Ci,l + Di - Ci - max_jobs*Ti)
        workload_task = min(workload_task,Dk - Ck + 1)
        total_workload += workload_task

        thresh = Ck + np.floor(total_workload/m)
        if Dk < thresh:
            return 0
    return 1

In [89]:
def deadline_analysis(ptaskset,m):    #taskset = {period,execution time,deadline}
    num_tasks = len(ptaskset)
    cleared = 0
    for k in range(num_tasks):
        
        total_workload = 0
        Ck,Dk = ptaskset[k].execution_time,ptaskset[k].deadline
        l = Dk
        for i in range(k):
            Ti,Ci,Di = ptaskset[i].period,ptaskset[i].execution_time,ptaskset[i].deadline
            max_jobs = np.floor(( l + Di - Ci )/Ti)
            workload_j = max_jobs*Ci + min(Ci,l + Di - Ci - max_jobs*Ti)
            workload_j = min(workload_j,Dk - Ck + 1)
            total_workload += workload_j

        thresh = Ck + np.floor(total_workload/m)
        if Dk >= thresh:
            cleared += 1

    return (cleared/num_tasks)*100

In [ ]:
def responsetimeanalysis(ptaskset,m):
    num_tasks = len(ptaskset)
    cleared = 0
    for k in range(num_tasks):
        Ck,Dk = ptaskset[k].execution_time,ptaskset[k].deadline
        l = Dk
        res = Ck
        new_res = Ck + 10
        if k == 1:
            if res <= Dk:
                continue
            else:
                return 0

        while new_res - res > 0.7 and new_res <= Dk :

            total_workload = 0
            for i in range(k):
                Ti,Ci = ptaskset[i].period,ptaskset[i].execution_time
                max_jobs = np.floor(( l + res - Ci )/Ti)
                workload_j = max_jobs*Ci + min(Ci,l + res - Ci - max_jobs*Ti)
                workload_j = min(workload_j,res - Ck + 1)
                total_workload += workload_j

            new_res = Ck + np.floor(total_workload/m)

        if new_res - res <= 0.7 and new_res <= Dk :
            cleared += 1

    return (cleared/num_tasks)*100

In [131]:
cores = 8
taskset = TaskSet(cores*5,0.6,10,100)
taskset.generate()
DMPO(taskset.tasks)
v = deadline_analysis(taskset.tasks,cores)
print(v)

100.0


In [103]:
from matplotlib import pyplot as plt

In [134]:
cores = 8
ut = np.linspace(0.1,cores,20)
sch = []
for i in ut:
    avg = 0
    for j in range(1000):
        #print(j)
        taskset = TaskSet(cores*5,i,10,100)
        taskset.generate()
        DMPO(taskset.tasks)
        v = deadline_analysis(taskset.tasks,cores)
        avg += v
    sch.append(avg/1000)
plt.plot(ut,sch,label = 'DMPO') 
sch = []
for i in ut:
    avg = 0
    for j in range(1000):
        #print(j)
        taskset = TaskSet(cores*5,i,10,100)
        taskset.generate()
        D_CMPO(taskset.tasks)
        v = deadline_analysis(taskset.tasks,cores)
        avg += v
    sch.append(avg/1000)


plt.plot(ut,sch,label = 'D_CMPO') 

sch = []
for i in ut:
    avg = 0
    for j in range(1000):
        taskset = TaskSet(cores*5,i,10,100)
        taskset.generate()
        DkC(taskset.tasks,cores)
        v = deadline_analysis(taskset.tasks,cores)
        avg += v
    sch.append(avg/1000)

plt.plot(ut,sch,label = 'DkC') 

plt.legend()
plt.show()

KeyboardInterrupt: 